# Train custom DeepD3 model

## Imports

In [1]:
from pathlib import Path
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, CSVLogger,
    LearningRateScheduler)
import segmentation_models as sm
import time
from datetime import datetime
from deepd3.model import DeepD3_Model
from deepd3.training.stream import DataGeneratorStream
from deepd3.training.utils import schedule, set_seed

Segmentation Models: using `keras` framework.


Ensure GPU is detected.

In [2]:
assert tf.config.list_physical_devices('GPU')
assert tf.test.is_built_with_cuda()

sm.set_framework("tf.keras")
gpus = tf.config.experimental.list_physical_devices('GPU')

## Define training-related variables

In [3]:
set_seed(41)

# Training parameters
batch_size = 32 
filters = 32
learning_rate = 5e-4
epochs = 30

Random seed 41 has been set.


## Define path variables

In [4]:
# today = datetime.now().strftime("%y%m%d")
today = "251215"

# Datasets
d3set_folder = Path(r"../data/d3set")
training_data_path = d3set_folder / f"{today}_training_dataset.d3set"
validation_data_path = d3set_folder / f"{today}_validation_dataset.d3set"

# Model
model_folder = Path(r"../data/models")
model_folder.mkdir(parents=True, exist_ok=True)
model_filename = f"model_{today}.h5"
model_output_path = model_folder / model_filename

# Logger
logger_filename = model_folder / f'model_{today}_logger.csv'

In [11]:
from deepd3.data_preparation.analyze_labels import suggest_min_content

# Get recommended min_content based on dataset label density
recommended_min_content = suggest_min_content(
    training_data_path,
    patch_size=128,
    num_samples=10000,
    percentile=10
)

recommended_min_content_val = suggest_min_content(
    validation_data_path,
    patch_size=128,
    num_samples=10000,
    percentile=10
)

print(f"Recommended min_content: {recommended_min_content}")
print(f"Recommended min_content (val): {recommended_min_content_val}")
min_content = recommended_min_content

Recommended min_content: 304
Recommended min_content (val): 294


## Load training data

Given mixed resolution during training, set ```target_resolution``` to ```None```.

In [6]:
# Create data generators for training and validation
dg_training = DataGeneratorStream(
    training_data_path,
    batch_size=batch_size,
    target_resolution=None,
    min_content=min_content)

dg_validation = DataGeneratorStream(
    validation_data_path,
    batch_size=batch_size,
    target_resolution=None,
    min_content=min_content,
    augment=False,
    shuffle=False)

# Sanity check data generators
# X, Y = dg_training[0]
# print(X.shape, Y[0].shape)

## Creating model

In [7]:
# Create a naive DeepD3 model with a given base filter count
model = DeepD3_Model(
    filters=filters,
    input_shape=(None, None, 1))

model.compile(
    Adam(learning_rate=learning_rate),
    [sm.losses.dice_loss, "mse"],
    metrics=['acc', sm.metrics.iou_score])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, None,  0           []                               
                                 1)]                                                              
                                                                                                  
 enc_layer0_conv1 (Conv2D)      (None, None, None,   288         ['input[0][0]']                  
                                32)                                                               
                                                                                                  
 enc_layer0_conv1_BN (BatchNorm  (None, None, None,   128        ['enc_layer0_conv1[0][0]']       
 alization)                     32)                                                           

Loading some training callbacks, such as adjusting the learning rate across time, saving training progress and intermediate models

In [8]:
# Save best model automatically during training
mc = ModelCheckpoint(
    model_output_path,
    save_best_only=True)

# Save metrics
csv = CSVLogger(logger_filename)

# Adjust learning rate during training to allow for better convergence
lrs = LearningRateScheduler(schedule)

## Train DeepD3 model

In [9]:
from tqdm.keras import TqdmCallback
start_time = time.time()
print(
    "Training started at: "
    f"{datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S')}")

# Actually train the network
history = model.fit(
    dg_training,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=dg_validation,
    callbacks=[mc, csv, lrs, TqdmCallback(verbose=1)],
    verbose=0)

end_time = time.time()

print(
    "Training ended at: "
    f"{datetime.fromtimestamp(end_time).strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total training time: {(end_time - start_time) / 60:.2f} minutes")


Training started at: 2025-12-15 15:07:49


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Training ended at: 2025-12-15 16:58:58
Total training time: 111.15 minutes
Training ended at: 2025-12-15 16:58:58
Total training time: 111.15 minutes


## Save model and logger

In [10]:
model.save(model_output_path)

history_df = pd.DataFrame(history.history)
history_df.to_csv(logger_filename, index=False)